# Assess

## `df_archive`

#### Visual assessment
I visually assess `df_archive` by opening the csv file on Google Sheet. First thing I noticed there are a lot of empty field at the columns `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`. These fields indicate the tweet is a retweet or not. Since I only need original tweet data, I will need to drop rows with non-empty field under these columns for the purpose of this analysis. 

Next for the last four columns (`doggo`, `floofer`, `pupper` and  `puppo`), they all represent various stages of dogs so it should be grouped under a single column (untidy data).

Another data quality issue was found under `name` column. There are many invalid names called 'a'.

`source` column has no use in the analysis.

#### Programmatic assessment

1. Missing data: Missing data for columns related to retweet. (.info())
2. Data type: Erroneous datatypes (`timestamp`, `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`)
3. Duplicate data: no duplicate tweet. sum(df_archive.tweet_id.duplicated())
4. Abnormal outliers: Some exceptionally high rating in `rating_numerator`.(.describe()) Mean ratings is 13.1 but max rating is 1776 and some well over serval hundreds. Those are valid ratings, typical for [special occasions](https://twitter.com/dog_rates/status/749981277374128128/photo/1) or images [containing more than one dog](https://twitter.com/dog_rates/status/731156023742988288).

## `df_image`

#### Visual assessment

By visual assessing df_image, I notice there are multiple predictions for each dog. For simplicity, I will only take the most confident dog prediction(`p1` and `p1_dog`=`True`) and won't bother with the rest.

#### Programmatic assessment

1. Missing data: no missing data using .info()
2. Data type: Erroneous datatype for `tweet_id` (.info())
3. Duplicate data: no duplicate tweet (sum(df_image.tweet_id.duplicated())=0)
4. Data Validity: All numeric data are in their valid range (.describe())

### `df_tweet`

#### Visual assessment

Dataset looks good.

#### Programmatic assessment

1. Missing data: no missing data using .info()
2. Data type: Erroneous datatype for `id` (.info())
3. Duplicate data: no duplicate tweet (sum(df_image.tweet_id.duplicated())=0)
4. Data Validity: All numeric data are in their valid range (.describe())

Below concludes the issues found out from the three DataFrames.

### Quality
##### `df_archive`
- For `name` column, some are of the string "none" and some are named "a"
- "none" instead of empty under `doggo`, `floofer`, `pupper` and  `puppo`
- Erroneous datatypes (`tweet_id`, `timestamp`, `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`)
- 181 tweets are retweet (non-empty `retweeted_status_id`)
- `source` column has no use in the analysis.

##### `df_image` 
- Irrelevant data except `p1` and `p1_dog == True`
- Erroneous datatype `tweet_id`

#### `df_tweet`
- Erroneous datatype `id`

### Tidiness
- `df_image` should be part of `df_archive` 
- `df_tweet` should be part of `df_archive`

##### `df_archive` 
- One variable in four columns (`doggo`, `floofer`, `pupper` and  `puppo`).

#### `df_image`
- `df_image`: Multiple predictions regarding dog's type (`p1`, `p2`, `p3`)

# Clean

I will clean tidiness issue first. First, I made a copy of all DataFrame.

### **One variable in four columns (`doggo`, `floofer`, `pupper` and  `puppo`)**

*Define: Melt these four columns into a single column called `dog_stage`*

Since some tweets contain multiple dogs stages (there are more than two dogs in the same picture), I would only keep the data for tweet containing one stage and change to 'none' for tweets with multiple stages.

Steps:
- Melt `doggo`, `floofer`, `pupper` and  `puppo` into one column `dog_stage` using pd.melt()
- One tweet will have four rows, with information about each stage (none or not none)
- If dog contains two or more stage, change all to 'none'
- If all stages are none, remove any three of the entries
- If one stage is not none, remove other three entries with 'none'

After cleaning, `df_archive_clean` now has 2356 unique tweets with 366 tweets non "None" dog stages.

### **`df_image`: Multiple predictions regarding dog's type (`p1`, `p2`, `p3`)**

Use .drop() to drops `['p2', 'p2_conf', 'p2_dog', 'p3', 'p3_dog', 'p3_conf']`.

### **Merge `df_image_clean` with `df_archive_clean` by tweet_id**

*Define: Merge `df_image_clean` with `df_archive_clean` by `tweet_id`*.
After merging, `p1_dog` convert to type `object`. Need to change it back to type `bool`

### **`df_tweet` should be part of `df_archive`**

*Define: Merge `df_tweet_clean` with `df_archive_clean` by `tweet_id`*

First, change the column `id` in `df_tweet_copy` to `tweet_id` to match the column name of `df_archive_copy`. After merging, `favorite_count` and `retweet_count` are converted to float as there are some missing values. Need to convert to int by Pandas `Int64` 

# Quality

### **`df_archive`: 181 tweets are retweet (non-empty retweeted_status_id)**

*Define: Remove all retweet entries. Delete all retweet related columns afterward.*

2175 tweets remains after retweets deletion.

Then I delete `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, `in_reply_to_status_id`, `in_reply_to_user_id` columns since they no longer relevant to the analysis.

### **Erroneous datatypes (`tweet_id`, `timestamp`)**

*Define Change `tweet_id` to type `object`. Change `timestamp` to type `datetime`*

### **For `name` column, some are of the string "none" and some are named "a"**

*Define: Change the name "a" to "none".*

### `source` column has no use in the analysis.

*Define:* Drop `source` column

After cleaning the dataset, I store the dataset into a new csv called "we_rate_dog_clean.csv" using .to_csv()